In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"assets\Eikon\asia_data.csv", low_memory=False)
df

,Instrument,TotAssets.Date,Price Close.Date,MktCap.Date,CompanySharesOutstanding.Date,Company Name,Company Shares,Country of Incorporation,Country of Exchange,Country of Headquarters,...,Loans & Receivables - Total,Net Cash Flow from Operating Activities,Income Taxes - Payable - Short-Term,Trade Account Payables - Total,Advertising Expense,Labor & Related Expenses - Total,Dividend Per Share - Actual,"EPS - Basic - excl Extraordinary Items, Common - Total",CO2 Estimation Method,Estimated CO2 Equivalents Emission Total
0,UNIB.XA,NaN,NaN,NaN,NaN,Unibank OJSC,NaN,Armenia,Armenia,Armenia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HEZB.XA,NaN,NaN,NaN,NaN,Hayastani ekonomikayi zargatsman bank BBE,NaN,Armenia,Armenia,Armenia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AHEK.XA,NaN,NaN,NaN,NaN,Artsakh HEK BBE,NaN,Armenia,Armenia,Armenia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ESHG.XA,NaN,NaN,NaN,NaN,Yerevan Champagne Wines Factory OJSC,NaN,Armenia,Armenia,Armenia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EVG1.XA,NaN,NaN,NaN,NaN,Yerevan Jewellery Plant 1 Gnomon OJSC,NaN,Armenia,Armenia,Armenia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793513,IJC.HM,2001-12-31T00:00:00Z,NaN,2001-12-31T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793514,IJC.HM,2000-12-31T00:00:00Z,NaN,2000-12-31T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793515,IJC.HM,1999-12-31T00:00:00Z,NaN,1999-12-31T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793516,IJC.HM,1998-12-31T00:00:00Z,NaN,1998-12-31T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
[c for c in df.columns]

['Instrument',
 'TotAssets.Date',
 'Price Close.Date',
 'MktCap.Date',
 'CompanySharesOutstanding.Date',
 'Company Name',
 'Company Shares ',
 'Country of Incorporation',
 'Country of Exchange',
 'Country of Headquarters',
 'Instrument Type',
 'ISIN',
 'ISIN Code',
 'ISIN_CODE',
 'SEDOL',
 'CUSIP Code',
 'SIC Industry Group Code',
 'SIC Industry Group Name',
 'SIC Industry Code',
 'SIC Industry Name',
 'GICS Sector Code',
 'GICS Sector Name',
 'GICS Industry Group Code',
 'GICS Industry Group Name',
 'GICS Industry Code',
 'GICS Industry Name',
 'GICS Sub-Industry Code',
 'GICS Sub-Industry Name',
 'CO2 Equivalent Emissions Direct, Scope 1',
 'CO2 Equivalent Emissions Indirect, Scope 2',
 'Upstream scope 3 emissions Purchased goods and services',
 'Upstream scope 3 emissions Capital goods',
 'Upstream scope 3 emissions Fuel- and Energy-related Activities',
 'Upstream scope 3 emissions Transportation and Distribution',
 'Upstream scope 3 emissions Waste Generated in Operations',
 'Upstr

In [4]:
df[df['Estimated CO2 Equivalents Emission Total'].notna()]

,Instrument,TotAssets.Date,Price Close.Date,MktCap.Date,CompanySharesOutstanding.Date,Company Name,Company Shares,Country of Incorporation,Country of Exchange,Country of Headquarters,...,Loans & Receivables - Total,Net Cash Flow from Operating Activities,Income Taxes - Payable - Short-Term,Trade Account Payables - Total,Advertising Expense,Labor & Related Expenses - Total,Dividend Per Share - Actual,"EPS - Basic - excl Extraordinary Items, Common - Total",CO2 Estimation Method,Estimated CO2 Equivalents Emission Total
9,ALBH.BH,2022-12-31T00:00:00Z,2022-12-29T00:00:00Z,2022-12-31T00:00:00Z,2022-12-29T00:00:00Z,Aluminium Bahrain BSC,1.420000e+09,Bahrain,Bahrain,Bahrain,...,6.493607e+08,1.301030e+09,NaN,5.631565e+07,NaN,2.615605e+08,0.225500,0.779596,Reported,9091290.000
10,ALBH.BH,2021-12-31T00:00:00Z,2021-12-30T00:00:00Z,2021-12-31T00:00:00Z,2021-12-30T00:00:00Z,NaN,1.420000e+09,NaN,NaN,NaN,...,8.351214e+08,7.066380e+08,NaN,8.710439e+07,NaN,3.267739e+08,0.071628,0.847598,Reported,9312720.000
35,ZAIN.BH,2022-12-31T00:00:00Z,2022-12-26T00:00:00Z,2022-12-31T00:00:00Z,2022-12-29T00:00:00Z,Zain Bahrain BSC,3.680000e+08,Bahrain,Bahrain,Bahrain,...,6.916446e+07,4.941316e+07,NaN,2.757294e+07,NaN,2.285362e+07,NaN,0.041648,Reported,46002.400
61,EBRIT.BH,2022-12-31T00:00:00Z,2022-12-13T00:00:00Z,2022-12-31T00:00:00Z,2022-12-29T00:00:00Z,Eskan Bank Realty Income Trust,1.980000e+08,Bahrain,Bahrain,Bahrain,...,6.001857e+04,1.968770e+06,NaN,6.599894e+05,NaN,NaN,NaN,0.004722,Median,229.684
243,AUBB.BH,2022-12-31T00:00:00Z,2022-08-17T00:00:00Z,2022-12-31T00:00:00Z,2022-12-29T00:00:00Z,Ahli United Bank BSC,1.114793e+10,Bahrain,Bahrain,Bahrain,...,2.149644e+10,1.779133e+09,NaN,NaN,NaN,1.863180e+08,NaN,0.045330,Median,6834.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792816,DXG.HM,2022-12-31T00:00:00Z,2022-12-30T00:00:00Z,2022-12-31T00:00:00Z,2022-12-30T00:00:00Z,Dat Xanh Group JSC,6.100315e+08,Vietnam,Vietnam,Vietnam,...,4.531424e+08,-1.627650e+08,3.325893e+07,4.868073e+07,NaN,NaN,0.000000,0.010478,Median,34631.000
792817,DXG.HM,2021-12-31T00:00:00Z,2021-12-31T00:00:00Z,2021-12-31T00:00:00Z,2021-12-31T00:00:00Z,NaN,5.960256e+08,NaN,NaN,NaN,...,4.038497e+08,5.876511e+07,5.001512e+07,3.719473e+07,7.338794e+06,3.205479e+07,0.000000,0.084666,Median,29962.400
793076,KDH.HM,2022-12-31T00:00:00Z,2022-12-30T00:00:00Z,2022-12-31T00:00:00Z,2022-12-30T00:00:00Z,Khang Dien House Trading and Investment JSC,7.168300e+08,Vietnam,Vietnam,Vietnam,...,1.159574e+08,-4.403805e+07,1.809637e+07,4.613933e+06,8.445137e+05,5.186628e+06,0.000000,0.061665,Median,3443.170
793077,KDH.HM,2021-12-31T00:00:00Z,2021-12-31T00:00:00Z,2021-12-31T00:00:00Z,2021-12-31T00:00:00Z,NaN,6.429371e+08,NaN,NaN,NaN,...,9.615634e+07,-6.868984e+07,1.099164e+07,6.774489e+06,4.383966e+05,5.095140e+06,0.000000,0.071007,Median,4765.140


In [5]:
df['CO2 Estimation Method'].value_counts()

Reported    12394
Median       9269
Energy       1296
CO2           756
Name: CO2 Estimation Method, dtype: int64